In [229]:
import numpy as np
import pandas as pd
import scipy.spatial.distance as dt

In [230]:
self_point = [] # Self points all have radius R
detector = [] # Detector point coordinates
detector_radius = [] # Detector radius
test_point = [] # Test point coordinates
test_label = [] # Test point label- 0:True, 1&2:False
all_points = [] # All data points

In [231]:
# Hyperparameters of the NSA as specified by the instructions.txt file

R = 7 # Radius of self points
M = 30 # Number of detectors
samples = 25 # Number of test points
epochs = 30 # Number of epochs

In [232]:
# Handling the Training Data

train_in = pd.read_csv("input_train_data.csv", header=None)
train_out = pd.read_csv("output_train_data.csv", header=None)

In [233]:
train_in.drop(train_in.columns[0], axis=0, inplace=True)
train_in.drop(train_in.columns[0], axis=1, inplace=True)

train_in = train_in.astype(int)

In [234]:
c = p = 0
for i in range(len(train_in)):
    all_points.append(np.array(train_in.iloc[i])) # Appending all points of the training data to the all_points list
    if train_out.iloc[i][0] == 1:
        self_point.append(np.array(train_in.iloc[i])) # Appending all self points to the self_point list
        p += 1 # Counting the number of self points
    c += 1 # Counting the total number of points

print("Number of Self Points:", p, "\nNumber of Total Points:", c)

Number of Self Points: 32 
Number of Total Points: 100


In [235]:
# Handling the Test Data

test_out = pd.read_csv("output_test_data.csv", header=None)
test_in = pd.read_csv("input_test_data.csv", header=None)

test_in.drop(test_in.columns[0], axis=0, inplace=True)
test_in.drop(test_in.columns[0], axis=1, inplace=True)

test_in = test_in.astype(int)

In [236]:
c = 0
for i in range(len(test_in)): 
    c += 1 # Counting the total number of points
    test_point.append(np.array(test_in.iloc[i])) # Appending all test points to the test_point list
    if test_out.iloc[i][0] == 1: # Appending the label of the test points to the test_label list
        test_label.append(True)
    else:
        test_label.append(False)

In [237]:
dim = len(self_point[0]) # Dimension of the self data points

In [238]:
def distance(x1, x2): 
    '''
    Function to calculate the Minkowski distance between two points
    '''
    return dt.minkowski(x1, x2, 1) / len(x1) # Using p = 1 for the Minkowski distance

In [239]:
def calcR(x):
    '''
    Function to calculate the radius of the detector
    '''

    # Finding the distance of sample point nearest to the detector
    min_dist = float('inf') # Initializing the minimum distance to infinity
    for point in self_point:
        dist = distance(x, point) # Calculating the distance between the detector point and the self point
        if dist < min_dist: # Updating the minimum distance
            min_dist = dist

    # Checking if the point lies within the range of any detector
    pos = None # Initializing the position of the detector to None
    for i in range(len(detector)):
        dist = distance(x, detector[i]) # Calculating the distance between the detector point and the sample point
        if dist < detector_radius[i]: # Checking if the sample point lies within the range of the detector
            return -1 # Returning -1 if the sample point lies within the range of the detector
        if dist < min_dist: # Updating the minimum distance
            min_dist = dist
            pos = i # Updating the position of the detector

    # Calculating the radius. If it is a valid detector then r > 0
    if pos is not None:
        r = min_dist - detector_radius[pos] # Calculating the distance when detector point is near
    else:
        r = min_dist - R # Calculating the distance when detector point is far

    return r # Returning the radius

In [240]:
max_values = np.amax(all_points, axis=0) # Finding the maximum value of each field
max_values

array([25, 66, 44, 79])

In [241]:
class GA:
    '''
    Class to implement the Genetic Algorithm
    '''
    def __init__(self): # Initializing the class
        gene_T = [] 
        for i in range(dim): 
            gene_T.append(np.random.random_integers(0, max_values[i], size = samples)) # Generating a population of random points
        self.population = np.transpose(gene_T) # Transposing the gene matrix
        
        # Rectifying the generated population
        for p in range(samples):
            r = calcR(self.population[p]) # Calculating the radius of the possible detector point
            while r <= 0: # Rectifying until valid r > 0 is obtained
                i = 0
                while i < dim:
                    self.population[p][i] = np.random.randint(1, max_values[i])
                    i += 1
                r = calcR(self.population[p]) # Calculating the radius of the possible detector point

    def mutate(self, population): # Mutating the newly created population
        total_no = dim * int(samples / 2) # Total number of genes
        mutations = int(.4 * total_no) # 40% mutation is done
        arr = np.array([1] * mutations + [0] * (total_no - mutations)) # Creating an array of 1s and 0s
        np.random.shuffle(arr) # Shuffling the array
        arr = arr.reshape((int(samples / 2), dim)) # Reshaping the array
        indices = np.where(arr == 1) # Finding the places for mutations randomly
        for i in indices:
            population[i[0]][i[1]] = np.random.randint(1, max_values[i[1]]) # Mutating the population and providing the new values

        return population # Returning the mutated population
    
    def crossover(self, cost_list):
        '''
        Function to perform crossover
        '''
        median = np.median(cost_list) # Finding the median of the cost list
        new_population = np.zeros((int(samples / 2), dim), dtype = int) # Initializing the new population
        for i in range(int(samples / 2)):
            parent1 = self.population[i * 2] # Selecting the first parent
            parent2 = self.population[i * 2 + 1] # Selecting the second parent

            # Performing a multi-point crossover with odd places from parent2 and others from parent1
            for j in range(dim):
                if j % 2 == 0:
                    new_population[i][j] = parent1[j]
                else:
                    new_population[i][j] = parent2[j]

        new_population = self.mutate(new_population) # Mutating the new population

        # Rectifying the generated population
        for p in range(int(samples / 2)):
            r = calcR(new_population[p]) # Calculating the radius of the possible detector point
            while r <= 0: # Rectifying until valid r > 0 is obtained
                i = 0
                while i < dim:
                    new_population[p][i] = np.random.randint(1, max_values[i])
                    i += 1
                r = calcR(new_population[p]) # Calculating the radius of the possible detector point
                
        # Replacing the worst half of the population with the new population
        c = int(samples / 2) - 1 # Initializing the counter
        for i in range(len(self.population)): # Iterating over the population
            if cost_list[i] < median and c >= 0: # Checking if the cost is less than the median
                self.population[i] = new_population[c] # Replacing the worst half of the population with the new population
                c -= 1 # Decrementing the counter
            if c < 0: # Breaking the loop if the counter is less than 0
                break

In [242]:
def create_negative_detectors():
    '''
    Function to create the negative detectors
    '''
    obj = GA() # Creating an object of the GA class
    cost_list = [] # Initializing the cost list
    for i in range(epochs): # Iterating over the epochs
        cost_list = [] 
        for j in range(samples): 
            cost_list.append(calcR(obj.population[j])) # Calculating the radius of the detector
        obj.crossover(cost_list) # Performing crossover
        print(str(max(cost_list)) + " ", end = "")

    pos = np.arange(samples) # Creating an array of positions
    pos = [x for _, x in sorted(zip(cost_list, pos), reverse = True)] # Sorting the cost list in descending order
    pos = pos[:2] # Selecting the top 2 positions
    print("\nPopulation:\n",obj.population, "\n___________________________")
    return (obj.population[pos[0]], obj.population[pos[1]]), (cost_list[pos[0]], cost_list[pos[1]]) # Returning the top 2 positions

In [243]:
def populate_detectors():
    '''
    Function to populate the detectors
    '''
    i = 0
    while i < M: # Iterating over the number of detectors
        population, pop_rad = create_negative_detectors() # Creating the population of detectors

        # Appending the best detector to the detector list
        detector.append(population[0])
        detector_radius.append(pop_rad[0])
        i += 1
        print("INFO: Appending Detector " + str(i) + " " + " ".join(str(x) for x in population[0]) + " Radius: " + str(pop_rad[0])) 

        if i == M:
            break

        r = calcR(population[1]) # Calculating the r=new radius of the second best detector
        if r > 0: # Appending if valid
            detector.append(population[1])
            detector_radius.append(r)

            i += 1
            print("INFO: Appending Detector " + str(i) + " " + " ".join(str(x) for x in population[1]) + " Radius: " + str(r))


In [244]:
def calcDecR(x): 
    '''
    Function to check if the sample point lies within the range of any detector
    '''
    for i in range(M):
        dist = distance(detector[i], x) # Calculating the distance between the detector point and the sample point
        if dist < detector_radius[i]: # Checking if the sample point lies within the range of the detector
            return False # Returning False if the sample point lies within the range of the detector
    return True # Returning True if the sample point lies outside the range of the detector

In [1]:
def test():
    populate_detectors() # Populating the detectors
    acc = 0 # Initializing the accuracy
    for i in range(len(test_point)):
        pred = calcDecR(test_point[i]) # Calculating the prediction
        if pred == test_label[i]:
            acc += 1 # Incrementing the accuracy if the prediction is correct
            print("Correct Prediction " + " ".join(str(x) for x in test_point[i]) + " Predct: " + str(pred) + " Actual: " + str(test_label[i]))
        else:
            print("Wrong Prediction " + " ".join(str(x) for x in test_point[i]) + " Predct: " + str(pred) + " Actual: " + str(test_label[i]))
    print("Accuracy: " + str(acc / len(test_point))) # Printing the accuracy

In [2]:
test()

NameError: name 'populate_detectors' is not defined